In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.cuda.amp as amp
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import os

In [17]:
#df = pd.read_csv('/kaggle/input/no2-data/merged_no2_pixels.csv')
df = pd.read_csv('/kaggle/input/tempo-no2-data/merged_hcho_pixels.csv')
#df = pd.read_csv('/kaggle/input/tempo-no2-data/merged_o3_pixels.csv')

In [18]:
'''tmp=df
tmp ['day']= df['datetime']-df['datetime'].iloc[0]
tmp = tmp[['day', 'datetime']]
tmp'''
LAST_DATE=pd.to_datetime('2025-10-03')

### Clip outliers per pixel (IQR)


In [19]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)
LAST_DATE=pd.to_datetime(df.iloc[-1]['datetime'])
df

,datetime,0,1,2,3,4,5,6,7,8,...,36570,36571,36572,36573,36574,36575,36576,36577,36578,36579
0,2023-08-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-08-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-08-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,2025-09-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
757,2025-09-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
758,2025-10-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
759,2025-10-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
all_days = pd.date_range(df['datetime'].min().date(), df['datetime'].max().date(), freq="D")
df_full = df.set_index('datetime').reindex(all_days).reset_index()
df_full = df_full.rename(columns={'index': 'datetime'})

In [21]:
pixel_data = df.drop(columns=['datetime']).values
Q1 = np.percentile(pixel_data, 25, axis=0)
Q3 = np.percentile(pixel_data, 75, axis=0)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
pixel_data_capped = np.clip(pixel_data, lower, upper)
pixel_data_capped = np.clip(pixel_data_capped, 0, None)

df_capped = pd.DataFrame(pixel_data_capped, columns=df_full.columns[1:])
df_capped.insert(0, 'datetime', df_full['datetime'])
df=df_capped

In [22]:

# Interpolate missing values per pixel
df.iloc[:,1:] = df.iloc[:,1:].interpolate(limit_direction='both')
print("After filling gaps:", df.shape)

After filling gaps: (761, 36581)


### Scaling

In [23]:
raw_scaler = MinMaxScaler()
raw_scaler.fit(df.drop(columns=['datetime']))

scaler = MinMaxScaler()
pixel_data_scaled = scaler.fit_transform(df.drop(columns=['datetime']))

df_scaled = pd.DataFrame(pixel_data_scaled, columns=df.columns[1:])
df_scaled.insert(0, 'datetime', df['datetime'])

print("Scaling done, shape:", df_scaled.shape)

Scaling done, shape: (761, 36581)


In [24]:
H, W = 118, 310  # Original dimensions
#H, W = 122, 310  # Original dimensions

TARGET_H, TARGET_W = 256, 256  # Target dimensions for pre-trained model

# Working directories
WORK_DIR = Path('/kaggle/working/unet_forecast')
days_dir = WORK_DIR / 'days_npy'
meta_path = WORK_DIR / 'days_metadata.csv'

### Downsampling Stage

In [25]:
if days_dir.exists():
    print('Cleaning existing days directory...')
    for f in days_dir.glob('*.npy'):
        f.unlink()   # delete existing .npy files
else:
    days_dir.mkdir(parents=True)

Cleaning existing days directory...


In [26]:
dates = df_scaled['datetime']             # pandas Series of dates
data = df_scaled.drop(columns=['datetime']).values  # numpy array of pixel values

Compute how much padding is needed on each side so that (H, W) 

U-Net is divisible by 16 

In [27]:
# No padding needed since we're resizing to 256x256
print(f"Resizing images from {H}x{W} to {TARGET_H}x{TARGET_W}")

Resizing images from 118x310 to 256x256


In [28]:
import cv2

meta_rows = []
for i, d in enumerate(dates):
    arr = data[i].reshape(H, W)
    # Resize to 256x256 using OpenCV
    arr_resized = cv2.resize(arr, (TARGET_W, TARGET_H), interpolation=cv2.INTER_LINEAR)
    fpath = days_dir/f"day_{i:04d}.npy"
    np.save(fpath, arr_resized.astype(np.float32))
    meta_rows.append([d, fpath.as_posix(), H, W])  # Store original dimensions

meta_df = pd.DataFrame(meta_rows, columns=['datetime','file','original_h','original_w'])

meta_df.to_csv(meta_path, index=False)
meta_df
print(f"Saved {len(meta_df)} resized days to {days_dir}")

Saved 761 resized days to /kaggle/working/unet_forecast/days_npy


### Dataset Sequence

### Model

In [29]:
class DaySequenceDataset(Dataset):
    def __init__(self, meta_df, lookback_days):
        self.meta_df = meta_df.reset_index(drop=True)
        self.lookback_days = lookback_days
    def __len__(self):
        return len(self.meta_df) - self.lookback_days
    def __getitem__(self, idx):
        files = self.meta_df['file'].iloc[idx:idx+self.lookback_days].tolist()
        x = np.stack([np.load(f) for f in files], axis=0)  # Shape: (LOOKBACK_DAYS, 256, 256)
        y = np.load(self.meta_df['file'].iloc[idx+self.lookback_days])  # Shape: (256, 256)
        return torch.from_numpy(x).float(), torch.from_numpy(y).float()

LOOKBACK_DAYS = 7
train_size = int(len(meta_df)*0.90)
val_size = len(meta_df) - (LOOKBACK_DAYS+3)

train_meta = meta_df.iloc[:train_size]
val_meta = meta_df.iloc[train_size:]
test_meta =  meta_df.iloc[val_size:]

train_dataset = DaySequenceDataset(train_meta, LOOKBACK_DAYS)
val_dataset = DaySequenceDataset(val_meta, LOOKBACK_DAYS)
test_dataset = DaySequenceDataset(test_meta, LOOKBACK_DAYS)


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=3, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=3, pin_memory=True)

print(f"Train samples: {len(train_dataset)} Val samples: {len(val_dataset)}")

Train samples: 677 Val samples: 70


In [30]:
# Load pre-trained U-Net model from torch.hub
# Note: This model expects 3-channel input, so we'll need to adapt our data
print("Loading pre-trained U-Net model...")
model_hub = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

# Create a wrapper to handle single-channel to 3-channel conversion
class UNetWrapper(nn.Module):
    def __init__(self, pretrained_model, input_channels=7):
        super().__init__()
        self.pretrained_model = pretrained_model
        # Add a 1x1 conv to convert from LOOKBACK_DAYS channels to 3 channels
        self.channel_adapter = nn.Conv2d(input_channels, 3, kernel_size=1)
        
    def forward(self, x):
        # Convert from LOOKBACK_DAYS channels to 3 channels
        x_adapted = self.channel_adapter(x)
        # Pass through pre-trained U-Net
        return self.pretrained_model(x_adapted)

Loading pre-trained U-Net model...


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to /root/.cache/torch/hub/checkpoints/unet-e012d006.pt


In [31]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNetWrapper(model_hub, input_channels=LOOKBACK_DAYS).to(DEVICE)
print('Model params:', sum(p.numel() for p in model.parameters()))
print('Pre-trained U-Net loaded successfully!')

Model params: 7763065
Pre-trained U-Net loaded successfully!


### Training

In [ ]:
# Replace the existing training cell with this enhanced version

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=7e-6)
scaler_amp = amp.GradScaler()

EPOCHS = 80   
patience = 5            
best_val_loss = float('inf')
patience_counter = 0

print("Starting initial training phase...")
print("="*50)

for epoch in range(EPOCHS):
    # Training
    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} (train)")
    for xb, yb in loop:
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        optimizer.zero_grad()
        with amp.autocast():
            preds = model(xb)
            loss = criterion(preds, yb.unsqueeze(1))
        scaler_amp.scale(loss).backward()
        scaler_amp.step(optimizer)
        scaler_amp.update()
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(DEVICE, non_blocking=True)
            yb = yb.to(DEVICE, non_blocking=True)
            with amp.autocast():
                preds = model(xb)
                loss = criterion(preds, yb.unsqueeze(1))
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")

    # Early stopping check 
    if val_loss < best_val_loss - 1e-6:  
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), '/kaggle/working/best_model.pth')
    else:
        patience_counter += 1
        print(f"No improvement for {patience_counter} epoch(s)")
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

print(f"\nInitial training completed. Best validation loss: {best_val_loss:.6f}")

Starting initial training phase...


/tmp/ipykernel_36/2090717526.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_amp = amp.GradScaler()


Epoch 1/80 (train):   0%|          | 0/170 [00:00<?, ?it/s]

/tmp/ipykernel_36/2090717526.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/tmp/ipykernel_36/2090717526.py:41: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Epoch 1: train loss 0.0537, val loss 0.0389


Epoch 2/80 (train):   0%|          | 0/170 [00:00<?, ?it/s]

In [ ]:
# Fine-tuning Phase on Validation Data
print("\n" + "="*50)
print("Starting fine-tuning phase on validation data...")
print("="*50)

# Load the best model from initial training
model.load_state_dict(torch.load('/kaggle/working/best_model.pth', map_location=DEVICE))

# Create fine-tuning optimizer with lower learning rate
finetune_lr = 2e-6  # Lower learning rate for fine-tuning
finetune_optimizer = optim.Adam(model.parameters(), lr=finetune_lr)
finetune_scaler = amp.GradScaler()

# Fine-tuning parameters
FINETUNE_EPOCHS = 10
finetune_patience = 3
best_finetune_loss = float('inf')
finetune_patience_counter = 0

# Track losses for comparison
initial_val_loss = best_val_loss
finetune_losses = []

print(f"Fine-tuning with learning rate: {finetune_lr}")
print(f"Initial validation loss: {initial_val_loss:.6f}")

for epoch in range(FINETUNE_EPOCHS):
    model.train()
    running_finetune_loss = 0.0
    
    # Train on validation data (fine-tuning)
    loop = tqdm(val_loader, desc=f"Fine-tune Epoch {epoch+1}/{FINETUNE_EPOCHS}")
    for xb, yb in loop:
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        
        finetune_optimizer.zero_grad()
        with amp.autocast():
            preds = model(xb)
            loss = criterion(preds, yb.unsqueeze(1))
        
        finetune_scaler.scale(loss).backward()
        finetune_scaler.step(finetune_optimizer)
        finetune_scaler.update()
        
        running_finetune_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    
    avg_finetune_loss = running_finetune_loss / len(val_loader)
    finetune_losses.append(avg_finetune_loss)
    
    print(f"Fine-tune Epoch {epoch+1}: loss {avg_finetune_loss:.6f}")
    
    # Save best fine-tuned model
    if avg_finetune_loss < best_finetune_loss - 1e-7:
        best_finetune_loss = avg_finetune_loss
        finetune_patience_counter = 0
        torch.save(model.state_dict(), '/kaggle/working/finetuned_model.pth')
        print(f"  -> New best fine-tuned model saved (loss: {best_finetune_loss:.6f})")
    else:
        finetune_patience_counter += 1
        if finetune_patience_counter >= finetune_patience:
            print(f"  -> Fine-tuning early stopping after {epoch+1} epochs")
            break

# Summary of fine-tuning results
print(f"\nFine-tuning completed!")
print(f"Initial validation loss: {initial_val_loss:.6f}")
print(f"Best fine-tuned loss: {best_finetune_loss:.6f}")
improvement = initial_val_loss - best_finetune_loss
print(f"Improvement: {improvement:.6f} ({improvement/initial_val_loss*100:.2f}%)")

# Load the best fine-tuned model for subsequent evaluation
model.load_state_dict(torch.load('/kaggle/working/finetuned_model.pth', map_location=DEVICE))
print("Loaded best fine-tuned model for evaluation.")

In [ ]:
# Add this as a new cell to visualize the fine-tuning progress

# Visualize Fine-tuning Progress
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot fine-tuning loss progression
epochs_ft = range(1, len(finetune_losses) + 1)
ax1.plot(epochs_ft, finetune_losses, 'r-', linewidth=2, marker='o', markersize=4)
ax1.axhline(y=initial_val_loss, color='blue', linestyle='--', alpha=0.7, 
            label=f'Initial Val Loss: {initial_val_loss:.6f}')
ax1.axhline(y=best_finetune_loss, color='green', linestyle='--', alpha=0.7,
            label=f'Best Fine-tuned: {best_finetune_loss:.6f}')
ax1.set_xlabel('Fine-tuning Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Fine-tuning Progress')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Compare initial vs fine-tuned performance
models = ['Initial\n(Pre-trained)', 'Fine-tuned\n(Val Data)']
losses = [initial_val_loss, best_finetune_loss]
colors = ['blue', 'red']

bars = ax2.bar(models, losses, color=colors, alpha=0.7)
ax2.set_ylabel('Validation Loss')
ax2.set_title('Model Performance Comparison')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, loss in zip(bars, losses):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{loss:.6f}', ha='center', va='bottom', fontweight='bold')

# Add improvement percentage
if improvement > 0:
    improvement_pct = improvement/initial_val_loss*100
    ax2.text(0.5, max(losses)*0.5, f'Improvement:\n{improvement:.6f}\n({improvement_pct:.2f}%)', 
             ha='center', va='center', transform=ax2.transData,
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.show()

# Print final comparison
print("\n" + "="*50)
print("FINE-TUNING SUMMARY")
print("="*50)
print(f"Training Strategy: Pre-trained → Train on 85% → Fine-tune on Validation")
print(f"Fine-tuning Learning Rate: {finetune_lr}")
print(f"Fine-tuning Epochs: {len(finetune_losses)}")
print(f"Initial Model Loss: {initial_val_loss:.6f}")
print(f"Fine-tuned Model Loss: {best_finetune_loss:.6f}")
if improvement > 0:
    print(f"Performance Improvement: {improvement:.6f} ({improvement/initial_val_loss*100:.2f}%)")
else:
    print(f"Performance Change: {improvement:.6f} (No improvement)")
print("="*50)

In [ ]:
def resize_to_original(arr, original_h, original_w):
    """Resize from 256x256 back to original dimensions"""
    return cv2.resize(arr, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

# Load best model
model.load_state_dict(torch.load('/kaggle/working/best_model.pth', map_location=DEVICE))
model.eval()

print("Evaluating on test sequence...")
test_predictions = []
test_ground_truths = []

with torch.no_grad():
    for xb, yb in tqdm(test_loader, desc="Testing"):
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)

        # Get predictions
        preds = model(xb)

        # Store for later analysis (keep numpy arrays)
        test_predictions.append(preds.cpu().numpy())
        test_ground_truths.append(yb.cpu().numpy())

# Concatenate all batches along the batch dimension
all_test_preds = np.concatenate(test_predictions, axis=0)  # shape: (N, C, H, W)
all_test_gts = np.concatenate(test_ground_truths, axis=0)  # shape: (N, H, W) or (N, 1, H, W)

# Ensure ground truths have a channel dimension to match predictions
if all_test_gts.ndim == 3:
    all_test_gts = all_test_gts[:, np.newaxis, ...]  # (N, 1, H, W)

# Compute overall MSE and MAE across all pixels and samples (correct global averages)
diff = all_test_preds - all_test_gts
overall_test_mse = np.mean(diff ** 2)
overall_test_mae = np.mean(np.abs(diff))

print(f"\nTotal test samples evaluated: {all_test_preds.shape[0]}")
print("\nOverall Test Results:")
print(f"Test MSE: {overall_test_mse:.6f}")
print(f"Test MAE: {overall_test_mae:.6f}")


In [ ]:
# Test evaluation with detailed metrics
import warnings
warnings.filterwarnings('ignore')

# Load best model
model.load_state_dict(torch.load('/kaggle/working/best_model.pth', map_location=DEVICE))
model.eval()

print("Evaluating on test sequence...")
test_predictions_scaled = []
test_ground_truths_scaled = []
test_indices = []

with torch.no_grad():
    for i, (xb, yb) in enumerate(tqdm(test_loader, desc="Testing")):
        xb = xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        
        # Get predictions
        preds = model(xb)
        
        # Store for later analysis (keep numpy arrays on scaled data)
        test_predictions_scaled.append(preds.cpu().numpy())
        test_ground_truths_scaled.append(yb.cpu().numpy())
        test_indices.extend([val_size + LOOKBACK_DAYS + i*4 + j for j in range(len(xb))])

# Concatenate all batches
all_test_preds_scaled = np.concatenate(test_predictions_scaled, axis=0)  # (N, 1, 256, 256)
all_test_gts_scaled = np.concatenate(test_ground_truths_scaled, axis=0)    # (N, 256, 256)

# Ensure ground truths have channel dimension
if all_test_gts_scaled.ndim == 3:
    all_test_gts_scaled = all_test_gts_scaled[:, np.newaxis, ...]  # (N, 1, 256, 256)

print(f"Test predictions shape: {all_test_preds_scaled.shape}")
print(f"Test ground truths shape: {all_test_gts_scaled.shape}")

# Calculate metrics on scaled data
diff_scaled = all_test_preds_scaled - all_test_gts_scaled
mse_scaled = np.mean(diff_scaled ** 2)
mae_scaled = np.mean(np.abs(diff_scaled))

print(f"\nScaled Data Metrics:")
print(f"MSE (scaled): {mse_scaled:.6f}")
print(f"MAE (scaled): {mae_scaled:.6f}")

# Resize back to original dimensions and inverse transform to original scale
test_predictions_original = []
test_ground_truths_original = []

for i in range(len(all_test_preds_scaled)):
    # Resize from 256x256 back to original HxW
    pred_resized = resize_to_original(all_test_preds_scaled[i, 0], H, W)
    gt_resized = resize_to_original(all_test_gts_scaled[i, 0], H, W)
    
    # Flatten to 1D for inverse transform
    pred_flat = pred_resized.flatten().reshape(1, -1)
    gt_flat = gt_resized.flatten().reshape(1, -1)
    
    # Inverse transform using the scaler
    pred_original = scaler.inverse_transform(pred_flat).flatten()
    gt_original = scaler.inverse_transform(gt_flat).flatten()
    
    test_predictions_original.append(pred_original.reshape(H, W))
    test_ground_truths_original.append(gt_original.reshape(H, W))

# Convert to numpy arrays
test_predictions_original = np.array(test_predictions_original)
test_ground_truths_original = np.array(test_ground_truths_original)

print(f"Original scale predictions shape: {test_predictions_original.shape}")
print(f"Original scale ground truths shape: {test_ground_truths_original.shape}")

# Calculate metrics on original scale
diff_original = test_predictions_original - test_ground_truths_original
mse_original = np.mean(diff_original ** 2)
mae_original = np.mean(np.abs(diff_original))

print(f"\nOriginal Scale Metrics:")
print(f"MSE (original): {mse_original:.6f}")
print(f"MAE (original): {mae_original:.6f}")
print(f"RMSE (original): {np.sqrt(mse_original):.6f}")

print(f"\nTotal test samples evaluated: {len(test_predictions_original)}")

In [ ]:
# Forecast next 7 days after the data range
print("Generating 7-day forecast after the data range...")

# Get the last LOOKBACK_DAYS from the dataset for forecasting
last_sequence_files = meta_df['file'].iloc[-LOOKBACK_DAYS:].tolist()
print("days: ",last_sequence_files)
forecast_input = np.stack([np.load(f) for f in last_sequence_files], axis=0)  # Shape: (7, 256, 256)
forecast_input_tensor = torch.from_numpy(forecast_input).float().unsqueeze(0).to(DEVICE)

print(f"Forecast input shape: {forecast_input_tensor.shape}")

# Generate 7-day forecast
forecast_predictions_scaled = []
current_input = forecast_input_tensor.clone()

with torch.no_grad():
    for day in range(7):
        # Predict next day
        next_day_pred = model(current_input)
        forecast_predictions_scaled.append(next_day_pred.cpu().numpy()[0, 0])  # Remove batch and channel dims
        
        # Update input sequence: remove oldest day, add predicted day
        next_day_pred_expanded = next_day_pred.squeeze(0)  # Remove batch dim, keep channel dim
        current_input = torch.cat([current_input[:, 1:], next_day_pred_expanded.unsqueeze(0)], dim=1)

forecast_predictions_scaled = np.array(forecast_predictions_scaled)  # Shape: (7, 256, 256)
print(f"Forecast predictions shape: {forecast_predictions_scaled.shape}")

# Convert forecasts to original scale
forecast_predictions_original = []

for i in range(7):
    # Resize from 256x256 back to original HxW
    forecast_resized = resize_to_original(forecast_predictions_scaled[i], H, W)
    
    # Flatten and inverse transform
    forecast_flat = forecast_resized.flatten().reshape(1, -1)
    forecast_original = scaler.inverse_transform(forecast_flat).flatten()
    forecast_predictions_original.append(forecast_original.reshape(H, W))

forecast_predictions_original = np.array(forecast_predictions_original)

# Generate forecast dates - Start from the user-defined LAST_DATE variable
forecast_dates = pd.date_range(start=LAST_DATE + pd.Timedelta(days=1), periods=7, freq='D')

print(f"Starting forecast from: {LAST_DATE}")
print(f"Forecast generated for dates: {forecast_dates[0].date()} to {forecast_dates[-1].date()}")
print(f"Forecast shape (original scale): {forecast_predictions_original.shape}")

# Summary statistics for forecast
print(f"\nForecast Summary (original scale):")
print(f"Min value: {forecast_predictions_original.min():.6f}")
print(f"Max value: {forecast_predictions_original.max():.6f}")
print(f"Mean value: {forecast_predictions_original.mean():.6f}")
print(f"Std value: {forecast_predictions_original.std():.6f}")

In [ ]:
# 1. Test Results Comparison
plt.rcParams['figure.figsize'] = (20, 15)

# Calculate proper aspect ratio and figure dimensions based on H, W
aspect_ratio = W / H  # 310 / 118 ≈ 2.63
img_height = 4  # Base height for individual images
img_width = img_height * aspect_ratio  # Width scaled by aspect ratio

# Get the last 3 days from the user-defined LAST_DATE variable for display
test_display_dates = pd.date_range(end=LAST_DATE, periods=3, freq='D')

fig, axes = plt.subplots(3, 4, figsize=(20, 12))
fig.suptitle('Test Predictions vs Ground Truth Comparison', fontsize=16)

# Show first 3 test samples with their corresponding dates
for i in range(min(3, len(test_predictions_original))):
    sample_date = test_display_dates[i].strftime('%Y-%m-%d')
    
    # Ground truth
    im1 = axes[i, 0].imshow(test_ground_truths_original[i], cmap='viridis', aspect='equal', origin='lower')
    axes[i, 0].set_title(f'Ground Truth - {sample_date}')
    axes[i, 0].axis('off')
    plt.colorbar(im1, ax=axes[i, 0], fraction=0.046, pad=0.04)
    
    # Prediction
    im2 = axes[i, 1].imshow(test_predictions_original[i], cmap='viridis', aspect='equal', origin='lower')
    axes[i, 1].set_title(f'Prediction - {sample_date}')
    axes[i, 1].axis('off')
    plt.colorbar(im2, ax=axes[i, 1], fraction=0.046, pad=0.04)
    
    # Absolute Error (MAE Map)
    abs_error_img = np.abs(test_predictions_original[i] - test_ground_truths_original[i])
    im3 = axes[i, 2].imshow(abs_error_img, cmap='Reds', aspect='equal', origin='lower')
    axes[i, 2].set_title(f'Absolute Error - {sample_date}')
    axes[i, 2].axis('off')
    plt.colorbar(im3, ax=axes[i, 2], fraction=0.046, pad=0.04)
    
    # Scatter plot
    axes[i, 3].scatter(test_ground_truths_original[i].flatten(), 
                       test_predictions_original[i].flatten(), 
                       alpha=0.5, s=1)
    axes[i, 3].plot([test_ground_truths_original[i].min(), test_ground_truths_original[i].max()], 
                    [test_ground_truths_original[i].min(), test_ground_truths_original[i].max()], 
                    'r--', linewidth=2)
    axes[i, 3].set_xlabel('Ground Truth')
    axes[i, 3].set_ylabel('Prediction')
    axes[i, 3].set_title(f'Correlation - {sample_date}')
    
    # Calculate correlation
    corr = np.corrcoef(test_ground_truths_original[i].flatten(), 
                       test_predictions_original[i].flatten())[0, 1]
    axes[i, 3].text(0.05, 0.95, f'R = {corr:.3f}', transform=axes[i, 3].transAxes, 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Print date information for clarity
print(f"\nTest Results Display Information:")
print(f"Last day (LAST_DATE): {LAST_DATE.strftime('%Y-%m-%d')}")
print(f"Test samples shown for dates: {test_display_dates[0].strftime('%Y-%m-%d')} to {test_display_dates[-1].strftime('%Y-%m-%d')}")

In [ ]:
# 3. Forecast Visualization - Individual Days
# Calculate proper figure size to maintain aspect ratio
single_img_width = 6 * aspect_ratio  # Width for single image
single_img_height = 6  # Height for single image

fig, axes = plt.subplots(2, 4, figsize=(24, 2 * single_img_height))
fig.suptitle('7-Day Forecast Results', fontsize=16)

# Show all 7 forecast days
for i in range(7):
    row = i // 4
    col = i % 4
    
    im = axes[row, col].imshow(forecast_predictions_original[i], cmap='viridis', aspect='equal', origin='lower')
    axes[row, col].set_title(f'Day +{i+1}: {forecast_dates[i].strftime("%Y-%m-%d")}', fontsize=12)
    axes[row, col].axis('off')
    plt.colorbar(im, ax=axes[row, col], fraction=0.046, pad=0.04)

# Remove the empty subplot (8th position)
fig.delaxes(axes[1, 3])

plt.tight_layout()
plt.show()

In [ ]:
# 4. Time Series Analysis - Historical Data + Forecasts
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Historical data (last 30 days) + forecasts
hist_days = 30
if len(meta_df) >= hist_days:
    hist_start_idx = len(meta_df) - hist_days
    hist_dates = pd.to_datetime(meta_df['datetime'].iloc[hist_start_idx:])
    hist_means = []
    
    for i in range(hist_start_idx, len(meta_df)):
        data_scaled = np.load(meta_df['file'].iloc[i])  # Shape: (256, 256)
        # Resize back to original dimensions
        data_resized = resize_to_original(data_scaled, H, W)
        data_original = scaler.inverse_transform(data_resized.flatten().reshape(1, -1)).flatten()
        hist_means.append(data_original.reshape(H, W).mean())
    
    # Plot historical + forecast means
    all_dates = list(hist_dates) + list(forecast_dates)
    all_means = hist_means + [day.mean() for day in forecast_predictions_original]
    
    axes[0].plot(hist_dates, hist_means, 'b-', linewidth=2, label='Historical', marker='o', markersize=4)
    axes[0].plot(forecast_dates, [day.mean() for day in forecast_predictions_original], 
                'r-', linewidth=2, label='Forecast', marker='s', markersize=4)
    axes[0].axvline(x=hist_dates.iloc[-1], color='gray', linestyle='--', alpha=0.7)
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Mean NO2 Value')
    axes[0].set_title('Time Series: Historical Data + 7-Day Forecast')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].tick_params(axis='x', rotation=45)

# Forecast uncertainty (std across spatial dimensions)
forecast_stds = [day.std() for day in forecast_predictions_original]
forecast_means = [day.mean() for day in forecast_predictions_original]

axes[1].plot(forecast_dates, forecast_means, 'r-', linewidth=2, marker='s', markersize=6, label='Mean')
axes[1].fill_between(forecast_dates, 
                     np.array(forecast_means) - np.array(forecast_stds),
                     np.array(forecast_means) + np.array(forecast_stds),
                     alpha=0.3, label='±1 Std')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('NO2 Value')
axes[1].set_title('7-Day Forecast with Spatial Uncertainty')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 5. Additional Forecast Visualizations - Side by Side Comparison
# Create a comprehensive comparison view
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Show first, middle, and last forecast day
forecast_indices = [0, 3, 6]  # Days 1, 4, and 7
forecast_labels = ['Day +1', 'Day +4', 'Day +7']

for idx, (f_idx, label) in enumerate(zip(forecast_indices, forecast_labels)):
    im = axes[idx].imshow(forecast_predictions_original[f_idx], cmap='viridis', aspect='equal', origin='lower')
    axes[idx].set_title(f'{label}: {forecast_dates[f_idx].strftime("%Y-%m-%d")}', fontsize=12)
    axes[idx].axis('off')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

plt.suptitle('Forecast Evolution: Selected Days', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 6. Final Results Summary
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
print(f"Data Dimensions: {H} × {W} pixels")
print(f"Aspect Ratio: {aspect_ratio:.2f}")
print(f"\nTest Set Performance:")
print(f"  • MSE (original scale): {mse_original:.6f}")
print(f"  • MAE (original scale): {mae_original:.6f}")
print(f"  • RMSE (original scale): {np.sqrt(mse_original):.6f}")
print(f"  • Correlation: {overall_corr:.3f}")
print(f"  • Number of test samples: {len(test_predictions_original)}")
print(f"\nForecast Summary:")
print(f"  • Forecast period: {forecast_dates[0].strftime('%Y-%m-%d')} to {forecast_dates[-1].strftime('%Y-%m-%d')}")
print(f"  • Mean forecast value: {forecast_predictions_original.mean():.6f}")
print(f"  • Forecast std: {forecast_predictions_original.std():.6f}")
print(f"  • Forecast range: [{forecast_predictions_original.min():.6f}, {forecast_predictions_original.max():.6f}]")
print(f"\nSpatial Statistics:")
print(f"  • Test correlation per sample: Min={np.array([np.corrcoef(test_ground_truths_original[i].flatten(), test_predictions_original[i].flatten())[0,1] for i in range(len(test_predictions_original))]).min():.3f}, Max={np.array([np.corrcoef(test_ground_truths_original[i].flatten(), test_predictions_original[i].flatten())[0,1] for i in range(len(test_predictions_original))]).max():.3f}")
print(f"  • Mean squared error per sample: Min={sample_mse.min():.6f}, Max={sample_mse.max():.6f}")
print("="*60)

In [ ]:
# Save 7-day forecast in original data format (datetime + flattened pixel columns)
print("Preparing forecast data in original format...")

# Create forecast dataframe
forecast_data_rows = []

for i, forecast_date in enumerate(forecast_dates):
    # Get the 2D forecast for this day (shape: H, W)
    forecast_2d = forecast_predictions_original[i]  # Shape: (118, 310)
    
    # Flatten to 1D (same as original data format)
    forecast_flattened = forecast_2d.flatten()  # Shape: (36580,) = 118*310
    
    # Create row: [datetime, pixel_0, pixel_1, pixel_2, ...]
    row = [forecast_date] + forecast_flattened.tolist()
    forecast_data_rows.append(row)

# Create column names (same as original data format)
num_pixels = H * W  # 118 * 310 = 36580
column_names = ['datetime'] + [str(i) for i in range(num_pixels)]

# Create DataFrame
forecast_df = pd.DataFrame(forecast_data_rows, columns=column_names)

# Display info about the forecast dataframe
print(f"Forecast DataFrame shape: {forecast_df.shape}")
print(f"Columns: datetime + {num_pixels} pixel columns (0 to {num_pixels-1})")
print(f"Date range: {forecast_df['datetime'].iloc[0].strftime('%Y-%m-%d')} to {forecast_df['datetime'].iloc[-1].strftime('%Y-%m-%d')}")

# Show first few rows and columns
print("\nFirst 3 rows and first 10 columns:")
print(forecast_df.iloc[:3, :10])

# Save to CSV
#forecast_output_path = '/kaggle/working/no2_forecast_7days.csv'
#forecast_output_path = "/kaggle/working/o3_forecast_7days.csv"
forecast_output_path = "/kaggle/working/hcho_forecast_7days.csv"

forecast_df.to_csv(forecast_output_path, index=False)
print(f"\nForecast saved to: {forecast_output_path}")

# Verify the saved file
print(f"Saved file shape: {pd.read_csv(forecast_output_path).shape}")
print("File successfully saved in original data format!")

In [ ]:
# Verify forecast data format and show sample statistics
print("="*60)
print("FORECAST DATA VERIFICATION")
print("="*60)

# Load the saved forecast to verify
saved_forecast = pd.read_csv(forecast_output_path)

print(f"Saved forecast shape: {saved_forecast.shape}")
print(f"Expected shape: (7, {num_pixels + 1})")  # 7 days, datetime + pixels
print(f"Shape matches expected: {saved_forecast.shape == (7, num_pixels + 1)}")

print(f"\nColumn names:")
print(f"First column (datetime): '{saved_forecast.columns[0]}'")
print(f"Pixel columns: {saved_forecast.columns[1]} to {saved_forecast.columns[-1]}")
print(f"Total pixel columns: {len(saved_forecast.columns) - 1}")

print(f"\nData sample (first day, first 10 pixels):")
first_day = saved_forecast.iloc[0]
print(f"Date: {first_day['datetime']}")
print("First 10 pixel values:", first_day[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']].values)

print(f"\nForecast statistics across all days and pixels:")
pixel_columns = saved_forecast.columns[1:]  # All columns except datetime
forecast_values = saved_forecast[pixel_columns].values.flatten()
print(f"Min value: {forecast_values.min():.6f}")
print(f"Max value: {forecast_values.max():.6f}")
print(f"Mean value: {forecast_values.mean():.6f}")
print(f"Std value: {forecast_values.std():.6f}")

print(f"\nDaily statistics:")
for i, row in saved_forecast.iterrows():
    day_values = row[pixel_columns].values
    print(f"Day {i+1} ({row['datetime']}): mean={day_values.mean():.6f}, std={day_values.std():.6f}")

print("="*60)
print("FORECAST DATA READY FOR USE!")
print("="*60)

In [ ]:
#df2=pd.read_csv("/kaggle/working/no2_forecast_7days.csv")
#df2=pd.read_csv("/kaggle/working/hcho_forecast_7days.csv")
df2=pd.read_csv("/kaggle/working/o3_forecast_7days.csv")
